# __WSI - ćwiczenie 1.__
### __Zagadnienie przeszukiwania i podstawowe podejścia do niego__


1. Narysować funkcje f(x) i g(x).
2. Zaimplementować algorytm najszybszego spadku oraz zastosować go do znalezienia minimum
funkcji f i g.
3. Zbadać wpływ rozmiaru kroku dla różnych (losowych) punktów początkowych.

In [2]:
import numpy as np
from plotly import graph_objs as go
import math
import pandas as pd

RNG = np.random.default_rng()

#### __Definicje funkcji oraz ich gradientów__

In [3]:
def f(vect):
    assert vect.shape == (1,)
    return 10*vect[0]**4 + 3*vect[0]**3 - 30*vect[0]**2 + 10*vect[0]
    
def g(vect):
    assert vect.shape == (2,)
    return 10*vect[1]**4 + 10*vect[0]**4 + 3*vect[0]**3 - 30*vect[0]**2 + 10*vect[0]

def grad_f(vect):
    assert vect.shape == (1,)
    return np.array(40*vect[0]**3 + 9*vect[0]**2 - 60*vect[0] + 10)

def grad_g(vect):
    assert vect.shape == (2,)
    return np.array([40*vect[0]**3 + 9*vect[0]**2 - 60*vect[0] + 10,
                     40*vect[1]**3])

### 

#### __Algorytm najszybszego spadku__


In [4]:
def gradient_descent(start_point, beta, grad, stop_treshlod, num_iters):
    """
    Implementation of gradient descent for minimalisation purposes,
    usable with n-dimensional loss functions, if a suitable gradient 
    function is provided.

    Args:
        start_point: coordinates of the first point
        beta: learning rate of the algorithm = step coefficient
        grad: a function which returns a matrix of partial derivatives of the input function
        stop_treshlod: minimal value of gradient
        num_iters: maximal number of iterations (steps)

    Returns:
        Returns a tuple of an array of points generated by alogrithm
        (dependiong on the gradiant function the point format may vary),
        and a boolean value which depicts whether the function has succesfully
        performed all of its iterations or reached given stop treshold. 

    Raises:
        None
    """
    steps = np.array([start_point])
    point = start_point
    for _ in range(num_iters):
        theta = grad(point)
        # check if algorithm reached a local minimum,
        # i.e. func has flattened out
        # Could be changed for Euclidean distance, 
        # but this should perform faster 
        if abs(theta.sum()) < stop_treshlod:
            break
        # prevent overflow errors
        if abs(theta.sum()) > 100000:
            return steps, False
        # perform algorithm step
        point = point - beta * theta
        # add point to the output array
        steps = np.append(steps, [point], 0)
    return steps, True


#### __Generowanie wykresów__


Wykres dla funkcji jednej zmiennej f(x):

In [5]:
max_r = 3
X = np.linspace(-max_r, max_r, 100)
Y = np.array([f(np.array([x])) for x in X])

pt = RNG.uniform(-max_r, max_r, 1)
dsc = gradient_descent(pt, 0.005, grad_f, 0.01, 100)
if not dsc[1]:
    print("Algorithm hasn't found the optimum, steps are out of bounds")
else:
    steps = np.array([x for x in dsc[0]
                    if abs(x[0]) < max_r])
    XS = steps[:, 0]
    YS = np.array([f(x) for x in steps])

    layout = go.Layout(width=700, height=500,
                       title='Gradient Descent of single variable function',
                       xaxis_title='x',
                       yaxis_title='f(x)',
                       plot_bgcolor='DarkSeaGreen')
    fig = go.Figure(data=[go.Scatter(x=X, y=Y, line=dict(color='DarkSlateGrey', width=3))], 
                    layout=layout)
    fig.add_trace(go.Scatter(x=XS, y=YS, mode='lines+markers', 
                            marker=dict(size=6, color=YS,
                            colorscale='Agsunset'),
                            line=dict(color='DarkSlateGrey', width=1)))
    fig.show()

Wykres dla funkcji dwóch zmiennych g(x, y):

In [6]:
max_r = 3
l = np.linspace(-max_r, max_r, 100)
X, Y = l, l
Z = np.array([[g(np.array([x, y])) for x in X] for y in Y])

pt = RNG.uniform(-max_r, max_r, 2)
dsc = gradient_descent(pt, 0.005, grad_g, 0.1, 100)
if not dsc[1]:
    print("Algorithm hasn't found the optimum, steps are out of bounds")
else:
    steps = np.array([pt for pt in dsc[0] if abs(pt[0]) < max_r and abs(pt[1]) < max_r])
    XS, YS = steps[:,0], steps[:,1]
    ZS = np.array([g(np.array([x, y])) for x, y in zip(XS, YS)])

    layout = go.Layout(width = 700, height =700,
                       title_text='Gradient Descent minimalisation of double variable function',
                       xaxis_title='x',
                       yaxis_title='y')                       
    fig = go.Figure(data=[go.Surface(x=X, y=Y, z=Z, colorscale='Emrld',
                                    opacity=0.5)], layout=layout)
    fig.update_traces(contours_z=dict(show=True, usecolormap=True))
    fig.add_scatter3d(x=XS, y=YS, z=ZS, mode='lines+markers', 
                      marker=dict(size=4, color=ZS,               
                                  colorscale='Agsunset'),
                      line=dict(color='DarkSlateGrey', width=0.7))
    fig.update_layout(scene=dict(zaxis_title="g(x, y)"))
    fig.show()

#### __Analiza wydajności funkcji najszybszego spadku w zależności od wartości współczynnika kroku__

Z przeprowadzonych obserwacji wywnioskowałem, że potrzebuję co najmniej kilkuset iteracji (uruchomień dla punktów początkowych) by zbadać generalne zachowanie algorytmu. Badania przeprowadzałem głównie na obszarze od -3 do 3, na którym 100 kroków algorytmu było zdecydowanie wystarczające żeby funkcja zatrzymała się po osiągnięciu dolnego limitu wartości gradientu, oznaczonego współczynnikiem odcięcia.


In [7]:
vals = [g(p) for p in dsc[0]]
layout = go.Layout(width=700, height=500,
                title_text='g(x,y) function value for GDA steps',
                plot_bgcolor='DarkSeaGreen',
                xaxis_title='Traversed point number',
                yaxis_title='g(x, y)')
fig = go.Figure(layout=layout)
fig.add_trace(go.Scatter(mode='lines+markers', 
                         marker=dict(size=6,
                                     color='DarkSlateGrey'),
                         line=dict(color='DarkSlateGrey', width=1), 
                         x=list(range(len(vals))), y=vals))
fig.show()

Współczynnik odcięcia __stop_treshold__ ustawiłem na 0.01, co wydało mi się wystarczające żeby stwierdzić że funkcja wystarczajaco zbliżyła się do minimum lokalnego i się stamtąd nie ruszy.

Funkcja testująca współczynnik kroku __beta__ uruchamia algorytm gradientowy __num_iters__ razy dla losowych punktów w zakresie badanego obszzaru i zwraca listę punktów końcowych wywołań algorytmu.

In [8]:
def test_beta(func, func_parameter_num, grad, beta: int, max_r, num_iters: int):
    end_points = []
    for _ in range(num_iters):
        p = RNG.uniform(max_r[0], max_r[1], func_parameter_num)
        dsc = gradient_descent(p, beta, grad, 0.01, 100)
        if not dsc[1]:
            end_points.append(np.nan)
        else:
            end_points.append(func(dsc[0][-1]))
    return np.array(end_points)

Metodą inżynierską zbadałem, że optymalna wartość zmienia się w zależnosci od rozimaru powierzchni dopuszczalnej. Dlatego, oraz dlatego, że w przypadkach innych funkcji których wykresów nie znamy, korzystne będzie przebadać większy zakres współczynników __beta__. W tym celu tworzę _dataframe_ który posłuży mi do dalszej analizy. Każda kolumna owej tabeli odpowiada badanym współczynnikom __beta__, a w wierszach znajdują się informacje o wartościach zwracanych przez _GDA_ dla losowych  punktów startowych.

In [9]:
b_arr = np.arange(0.0001, 0.02, 0.0001)
num_iters = 100
data = performance = {np.round(b, 4):(test_beta(g, 2, grad_g, b, [-10, 10], num_iters)) for b in b_arr}
df = pd.DataFrame(columns=[b for b in performance.keys()])
for col in df.columns:
    df[col] = performance[col]

In [10]:
df

,0.0001,0.0002,0.0003,0.0004,0.0005,0.0006,0.0007,0.0008,0.0009,0.0010,...,0.0190,0.0191,0.0192,0.0193,0.0194,0.0195,0.0196,0.0197,0.0198,0.0199
0,-26.269535,-39.662976,-41.343552,-41.788568,-42.072664,-6.944391,NaN,NaN,NaN,-42.486284,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12.316297,-39.072438,-5.725857,-41.763489,-42.069930,-42.243223,-6.705321,-42.436475,-42.462620,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-1.503612,-3.053182,-5.430724,-41.747649,-42.062335,-6.683283,-42.338352,-6.903593,-6.988688,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7.532371,-39.370664,-5.499900,-41.662056,-42.071238,-6.629198,-6.714096,-42.419627,-6.824759,-42.486126,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-6.934625,-3.398564,-41.980853,-6.176170,-42.140803,-42.241951,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-26.256864,-3.054697,-41.395723,-42.151014,-6.439463,-42.239875,NaN,NaN,-42.452936,-6.964782,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,12.190141,-39.031787,-6.933398,-6.120782,-6.701474,-6.645259,-42.342392,NaN,-6.831709,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,-33.273535,-40.654331,-6.943663,-41.820941,-42.085747,-6.617549,-42.327903,-42.564167,-6.828166,-42.492295,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7.005957,NaN,NaN
98,4.964770,-3.086365,-41.152552,-41.831556,-6.458235,-6.604457,-42.334708,NaN,-42.457791,-42.487236,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df.describe()

,0.0001,0.0002,0.0003,0.0004,0.0005,0.0006,0.0007,0.0008,0.0009,0.0010,...,0.0190,0.0191,0.0192,0.0193,0.0194,0.0195,0.0196,0.0197,0.0198,0.0199
count,100.000000,100.000000,100.000000,100.000000,100.000000,89.000000,72.000000,67.000000,64.000000,62.000000,...,2.000000,1.000000,2.000000,1.000000,5.000000,3.000000,0.0,5.000000,0.0,1.000000
mean,-9.040585,-21.249369,-24.945290,-27.246458,-28.345542,-23.244063,-25.624415,-28.524523,-26.833713,-31.004425,...,-19.498189,-7.006266,-32.199515,-41.461436,-23.673225,-24.931412,NaN,-17.787427,NaN,-3.267268
std,18.812742,18.242986,17.812967,17.673578,17.337665,17.763682,17.881640,17.662292,17.885864,16.797818,...,17.666719,NaN,0.358416,NaN,13.903949,17.792600,NaN,16.584937,NaN,NaN
min,-42.535578,-42.570996,-42.626092,-42.627673,-42.627661,-42.627652,-42.627549,-42.627668,-42.625914,-42.573405,...,-31.990446,-7.006266,-32.452953,-41.461436,-34.937674,-42.588227,NaN,-40.790291,NaN,-3.267268
25%,-27.212866,-39.308369,-41.162528,-41.809427,-42.105112,-42.243313,-42.351065,-42.441860,-42.461384,-42.495911,...,-25.744317,-7.006266,-32.326234,-41.461436,-33.510742,-33.894085,NaN,-30.059612,NaN,-3.267268
50%,-1.629420,-6.458591,-41.052900,-41.743471,-42.062104,-6.944391,-42.327530,-42.406840,-42.452963,-42.487079,...,-19.498189,-7.006266,-32.199515,-41.461436,-32.716796,-25.199942,NaN,-7.005973,NaN,-3.267268
75%,9.834957,-3.204489,-5.549151,-6.189222,-6.552396,-6.619661,-6.743222,-6.817122,-6.841744,-6.950853,...,-13.252061,-7.006266,-32.072795,-41.461436,-11.193882,-16.103004,NaN,-7.005957,NaN,-3.267268
max,12.345893,-3.025550,-1.310848,1.423025,-6.401347,-4.804068,-6.077547,0.453332,-2.756459,-6.122739,...,-7.005932,-7.006266,-31.946076,-41.461436,-6.007029,-7.006067,NaN,-4.075300,NaN,-3.267268


Następnie rysuję wykres wartości funkcji dla punktów zwracanych przez _GDA_, dla odpowiednich wartosci współczynników __beta__. 

In [12]:
layout = go.Layout(width=700, height=500,
                   title_text='Algorithm outputs in function of beta value',
                   plot_bgcolor='DarkSeaGreen',
                   xaxis_title='beta',
                   yaxis_title='min(g(x, y)) from GDA')
fig = go.Figure(layout=layout)
for _, row in df.iterrows():
    fig.add_trace(go.Scatter(mode='markers', marker_color='DarkSlateGrey', 
                             opacity=0.5, x=df.columns, y=row, showlegend = False))
fig.show()

Na tym wykresie nie widać jednak ile iteracji dla każdej wartości __beta__ nie osiągneło minimum. Wobec tego następny wykres przedstawia stosunek ilości nieudanych iteracji do wykonanych.

In [13]:
fail_rate = df.isna().sum()/num_iters

layout = go.Layout(width=700, height=500,
                title_text='Beta fail (out of bounds) rate',
                plot_bgcolor='DarkSeaGreen',
                xaxis_title='beta',
                yaxis_title='failed/total iterations')
fig = go.Figure(data=[go.Bar(x=df.columns, y=fail_rate, marker_color='DarkSlateGrey')], layout=layout)
fig.show()

Następnie po ustaleniu maksymalnego błędu wyznaczam najlepszy parametr __beta__, taki, który daje możliwie najniższe wyniki _GDA_.

In [14]:
max_error = 0.1
tmp_df = df[df.columns[df.isna().sum()/num_iters < max_error]]
best_b = tmp_df.columns[tmp_df.sum().argmin()]
print(best_b)

0.0005


Na koniec rysuję wykres wartości osiaganych przez _GDA_ dla znalezionego parametru __beta__. 

In [15]:
layout = go.Layout(width=700, height=500,
                   title_text='Best beta outputs',
                   plot_bgcolor='DarkSeaGreen',
                   xaxis_title='Iteration number',
                   yaxis_title='min(g(x, y)) from GDA')
fig = go.Figure(data=[go.Bar(y=df[best_b], marker_color='DarkSlateGrey')], layout=layout)
fig.update_yaxes(autorange="reversed")
fig.show()

Z powyższego wykresu można ostatecznie wnioskować, że funkcja ta ma dwa minima lokalne. Dla znalezionego parametru beta powinniśmy wobec tego otrzymać wartość minimalną w okolicy _-42_ bądź _-7_.

In [16]:
p = RNG.uniform(-10, 10, 2)
arr = gradient_descent(p, best_b, grad_g, 0.0001, 1000)[0]
g_min = arr[-1]
print(f"argmin(g) = {g_min}")
print(f"Step amount: {len(arr)-1}")
print(f"g(argmin) = {g(g_min)}")

argmin(g) = [1.01255865 0.15745625]
Step amount: 1000
g(argmin) = -7.0001754946103745


Analogiczny proces przeprowadziłem dla funkcji jednej zmiennej f(x):

In [17]:
b_arr = np.arange(0.0001, 0.02, 0.0001)
num_iters = 100
performance = {np.round(b, 4):(test_beta(f, 1, grad_f, b, [-10, 10], num_iters)) for b in b_arr}

df = pd.DataFrame(columns=[b for b in performance.keys()])
for col in df.columns:
    df[col] = performance[col]
max_error = 0.1
tmp_df = df[df.columns[df.isna().sum()/num_iters < max_error]]
best_b = tmp_df.columns[tmp_df.sum().argmin()]
print(best_b)

0.0004


In [18]:
p = RNG.uniform(-10, 10, 1)
arr = gradient_descent(p, best_b, grad_f, 0.0001, 1000)[0]
f_min = arr[-1]
print(f"argmin(f) = {f_min}")
print(f"Step amount: {len(arr)-1}")
print(f"f(argmin) = {f(f_min)}")

argmin(f) = [-1.41237125]
Step amount: 215
f(argmin) = -42.62767877894672


Ostatecznie, przy _10000_ iteracji dla funkcji _g(x, y)_ algorytm znalajduje minima: _g([-1.41, 0.05]) = -42.63_ oraz _g([1.01 0.05]) = -7.01_, a także dla funkcji f(x): _f([-1.41]) = -42.63_ i 
 _f([1.01]) = -7.01_, gdzie to pierwsze to minima globalne.

#### Wnioski


Ze względu na pewną ułomność tego algorytmu, oraz naturę badanych funkcji, algorytm ma tendencję wpadać w minima lokalne, a także wystrzeliwywać wyniki w kosmos kiedy pochodne badanych funkcji są duże oraz gwałtownie maleją w okolicy minimum. Najbezpieczniej wobec tego byłoby wybrać możliwie najmniejszy parametr __beta__ i możliwie najwięcej iteracji, co oczywiście nie jest możliwe jeśli zależny nam na czasie. Problem dotyczący wpadania w minima lokalne można próbować rozwiązać stosując stochastyczny algorytm spadku gradientu, który zamiast używać dokładnie obliczonego gradientu, korzysta z aproksymowanego gradientu który posiada przez to losowe zakłócenia. Ze względu na to że algorytm nie porusza się wtedy dokładnie w kierunku spadku wartości funkcji optymalizowanej, istnieje szansa, że lokalne minima zostaną ominięte. 